In [1]:

import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import os
import shutil
import pandas as pd
from scipy import signal

import sys
# Add the directory to the sys.path
sys.path.append('D:/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
import tools.CT_visualization_window as CT_visualization_window
import tools.loading_bar as loadingbar
%matplotlib qt

CT_visualization_window.check_existence()

you loaded the CT visualizer window v20.11.24


In [2]:
# ======= set working directories and files ==========

current_dir = os.getcwd()

parent_dir = os.path.dirname(current_dir)


input_file = parent_dir + "/034 - material edit/inp_material-c2_f0p6_MaterialEdited.inp"
output_file = current_dir + "/inp_material-c2_f0p6_MaterialEdited_remeshed.inp"

====================================================================================================================================================================================
========================================================================== detect flying elements ==================================================================================
====================================================================================================================================================================================

In [3]:
# ======= read inp file ==========

found_start = np.array([])
found_end = np.array([])

#open file
with open(input_file) as file:
    i=0
    for line in file:
        i += 1
        #look for the start of the elements definition
        if "*Element" in line:
            found_start = np.append(found_start,i)
        #look for the end of the elements definition
        if "*Elset" in line:
            found_end = np.append(found_end,i)

#the start is the first line that has the word "element" in it
starting_line = int(found_start[0])
#the end is the first line, after the line that start the elements definition, that has the word "elset" in it
ending_line = int(found_end[found_end>starting_line][0]-1)

#create a panda dictionnary where each line gives an element's ID-number and the ID-number of the 4 nodes that compose it...
#!!!!remember!!!!! If you use C3D10, there will be 10 nodes per element.. 4 corners and 6 edges. That is stupid to do.. Convert them to quadratic element after doing your island analysis, this will save you conputationnal cost...
elements_list = pd.read_csv(input_file, header=None, names = ["ElementNumber","Node1","Node2","Node3","Node4"], skiprows=starting_line, nrows=ending_line-starting_line)   #dictionary of the nodes of each element
print(elements_list.shape[0])


2191239


In [4]:
# create a seccond dictionnary that wil list for each node all the elements that it is connected to.
#remember: if you forgot to set the element type to linear, then you use C3D10 elements: 10 nodes per element.. 4 corners and 6 edges.

temp = pd.concat([elements_list[["ElementNumber",'Node1']].rename(columns={"Node1": "Node"}),
                  elements_list[["ElementNumber",'Node2']].rename(columns={"Node2": "Node"}),
                  elements_list[["ElementNumber",'Node3']].rename(columns={"Node3": "Node"}),
                  elements_list[["ElementNumber",'Node4']].rename(columns={"Node4": "Node"})
                  ])

if temp['ElementNumber'].shape[0] != 4 * elements_list['ElementNumber'].shape[0]:
    print('error')
else:
    print('good')

node_list = temp.groupby('Node')['ElementNumber'].apply(list).reset_index()

node_list['usage'] = node_list['ElementNumber'].apply(len)

node_list = node_list.sort_values('usage')          #dictionary of the elements of each node


good


In [5]:
print(node_list.shape[0]) #manually check in the inp file
#it should be 901685
print(elements_list.shape[0]) #manually check in the inp file
# it should be 4748068

442844
2191239


In [6]:
from collections import defaultdict

node_to_elements = defaultdict(list) #default dict is like a dict, but that allows you to specify a default value type for keys that don’t yet exist in the dictionary. list inside give the default value to be an empty list 
for _, row in node_list.iterrows():
    node_to_elements[row['Node']] = row['ElementNumber']

element_to_nodes = defaultdict(list) #default dict is like a dict, but that allows you to specify a default value type for keys that don’t yet exist in the dictionary. list inside give the default value to be an empty list 
k=0
for _, row in elements_list.iterrows():
    k+=1
    if k%100000==0:
        print(k)
    element_to_nodes[row['ElementNumber']] = [row['Node1'], row['Node2'], row['Node3'], row['Node4']]


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000


In [7]:
#============================== detect disconnected islands ===================================
# it's too long to check for every element if they are connected to every other..
#Instead I first get the nodes that are connected to few elements : they are most likely of the edge of the model.. There is no need to start for nodes that are in the middle of the model.
#Then I check if the are connected to "enough" elements to be "kinda sure" that they are not in an isolated element

# This works in this case because it is very easy to check if I missed something: I the resulting simulation doesn't work, I must have missed an island xD
from collections import defaultdict

low_connected_nodes = node_list[node_list['usage'] < 11]    #now I have a dictonary with all the node that are in 11 or less elements
connection_threshold = 1001  #limit at which a node is considered to not be in an island
print(f"{len(low_connected_nodes)} nodes are suspected, they will be actused if connected to less than {connection_threshold-1} other elements")

print(f"preprocessing variable, this should take around 2 minutes")
# Preprocess node_list for quick lookups
node_to_elements = defaultdict(list) #default dict is like a dict, but that allows you to specify a default value type for keys that don’t yet exist in the dictionary. list inside give the default value to be an empty list 
for _, row in node_list.iterrows():
    node_to_elements[row['Node']] = row['ElementNumber']

# Preprocess element_list for quick lookups
element_to_nodes = defaultdict(list) #default dict is like a dict, but that allows you to specify a default value type for keys that don’t yet exist in the dictionary. list inside give the default value to be an empty list 
for _, row in elements_list.iterrows():
    element_to_nodes[row['ElementNumber']] = [row['Node1'], row['Node2'], row['Node3'], row['Node4']]

island_elements = set()
progress = 0

#look at all the nodes that you defined as "suspect"
for node in low_connected_nodes['Node']:
    #start by assuming that this node is NOT in an island
    is_island = False

    #measuring progress
    loadingbar.main(progress, len(low_connected_nodes), bar_length=100)
    progress = progress + 1 

    # Get elements connected to the current node; transform it to a set for faster intersection check
    connected_elements = set(node_to_elements[node])   #this is faster than looking in a panda daaframe for the line at which the node is the good number (direct access instead of iterative search)  "node_list[node_list['Node'] == node]['ElementNumber']"
    
    #initialize the list of elements to which you looked for it's neighbors
    checked_elements = set()

    #until you find XX elements connected to your node...
    while len(connected_elements) < connection_threshold:
        unchecked_elements = connected_elements - checked_elements 
        
        #if there is no unchecked element
        if not unchecked_elements: 
            island_elements.update(connected_elements)  #kinda like append but for lists, it can add several elements
            is_island = True
            break
        
        # Check one of the unchecked elements
        element_to_check = unchecked_elements.pop()  #Remove a random item from the set

        #add it to the list of checked elements
        checked_elements.add(element_to_check)   #add only works for one element

        # Get nodes connected to this element
        element_to_check_nodes = element_to_nodes[element_to_check]

        #setup the "connection": number of nodes of this element that are connected to another element
        connections_per_element = 0
        for node_of_element in element_to_check_nodes:
            #if this node is connected to more than one element, then this element has one more connection
            if len(node_to_elements[node_of_element]) > 1:
                connections_per_element += 1

        # if in total, this element has only 1 or 2 nodes that are connected to other elements, skip it: they are not really connected, since elements connected by only 2 nodes can still rotate 
        if connections_per_element < 3:
            continue

        # Add new elements connected via these nodes
        for node_of_element in element_to_check_nodes:
            connected_elements.update(node_to_elements[node_of_element])

    if is_island:
        continue


island_elements = np.array(list(island_elements), dtype=np.uint64)
print(f"{len(island_elements)} elements are in an island")
print(island_elements)



62546 nodes are suspected, they will be actused if connected to less than 1000 other elements
preprocessing variable, this should take around 2 minutes
Progress: [####################################################################################################] 100%
216 elements are in an island
[ 179201  716805  716806  716807  716810  716811  716812  716813  716814
  716815  708624  369680  699410  699411  716817  716820  177174  716822
  716818  716821  716826  694309  369709  369711  369712  183364  524359
  401509  139394  168073  188553  444587  254128  698545  698548  698549
  698550  698551  698552  258241  239823  735440  364756  581873  515329
  515330  692488  692489  143628  692496  279885  414052  369678  381320
  414108  543146  676311  558552  481762  258546  281077  617979  255492
  189968  482887  618064  482902  245363  699007  634503  245386  385676
  738958  543374  275088  333459  245397  738967  738968  411315  145077
  570080  164579  147174  652015  652016  6

In [8]:
lines_to_delete = []
with open(input_file) as file:
    for i, line in enumerate(file):
        if i < starting_line:
            continue  # Skip lines before starting_line
        if i >= ending_line:
            break  # Stop after ending_line
        if int(line.split(',')[0]) in  island_elements:  # Adjusted for relative index
            lines_to_delete.append(i)
            #print(f'element {int(line.split(',')[0])} in line {i} should be deleted')

print(f"In total, {len(lines_to_delete)} elements will be deleted...")

In total, 216 elements will be deleted...


====================================================================================================================================================================================
========================================================================== detect flying nodes =====================================================================================
====================================================================================================================================================================================

In [16]:
# ======= read inp file ==========

found_start = np.array([])
found_end = np.array([])

#open file
with open(input_file) as file:
    i=0
    for line in file:
        i += 1
        #look for the start of the elements definition
        if "*Node" in line:
            found_start = np.append(found_start,i)
        #look for the end of the elements definition
        if "*Element" in line:
            found_end = np.append(found_end,i)

#the start is the first line that has the word "element" in it
starting_line = int(found_start[0])
#the end is the first line, after the line that start the elements definition, that has the word "elset" in it
ending_line = int(found_end[found_end>starting_line][0]-1)

#create a panda dictionnary where each line gives an element's ID-number and the ID-number of the 4 nodes that compose it...
#!!!!remember!!!!! If you use C3D10, there will be 10 nodes per element.. 4 corners and 6 edges. That is stupid to do.. Convert them to quadratic element after doing your island analysis, this will save you conputationnal cost...
original_nodes = pd.read_csv(input_file, header=None, names = ["NodeNumber","X","Y","Z"], skiprows=starting_line, nrows=ending_line-starting_line)   #dictionary of the nodes of each element
print(original_nodes.shape[0])

443304


In [17]:
node_set_connected_to_elements = set(node_list["Node"])
node_set_all_nodes = set(original_nodes["NodeNumber"])

nodes_to_delete = node_set_all_nodes-node_set_connected_to_elements
print(nodes_to_delete)

{4106, 40977, 40978, 157719, 151577, 151586, 157732, 127014, 118823, 157734, 157735, 157736, 421929, 94252, 18485, 405559, 108603, 26684, 79934, 26687, 182340, 237653, 79958, 153689, 159834, 153691, 235609, 147552, 131170, 153699, 131173, 153702, 153703, 223336, 36977, 366710, 26758, 196746, 186507, 127118, 211090, 69781, 178328, 114847, 114852, 118952, 59562, 59563, 170156, 2223, 155834, 172221, 157899, 194765, 47311, 26841, 366813, 26846, 141537, 141538, 63724, 157939, 63732, 401652, 55542, 55543, 151800, 55545, 45310, 45311, 88320, 147718, 96521, 151824, 63765, 88345, 88346, 164127, 131362, 88359, 43304, 74029, 74030, 74032, 74033, 125233, 86323, 364859, 172349, 102731, 147790, 164178, 158041, 156007, 156008, 156009, 344443, 39294, 53630, 39298, 395650, 39302, 70024, 80264, 39309, 104852, 8603, 217499, 51621, 108977, 129458, 129459, 6580, 129472, 129473, 70084, 59850, 129489, 70099, 129493, 84438, 129494, 129495, 129496, 129497, 129498, 129500, 410093, 299503, 178675, 410099, 12794,

In [18]:

with open(input_file) as file:
    for i, line in enumerate(file):
        if i < starting_line:
            continue  # Skip lines before starting_line
        if i >= ending_line:
            break  # Stop after ending_line
        if int(line.split(',')[0]) in  nodes_to_delete:  # Adjusted for relative index
            lines_to_delete.append(i)
            #print(f'element {int(line.split(',')[0])} in line {i} should be deleted')

print(f"In total, {len(nodes_to_delete)} nodes will be deleted...")

In total, 460 nodes will be deleted...


====================================================================================================================================================================================
========================================================================== create new input file ===================================================================================
====================================================================================================================================================================================

In [9]:
with open(input_file, 'r') as file, open(output_file, 'w') as new_file:
    for i, line in enumerate(file, start=0):
        if i not in lines_to_delete:  # Skip multiple lines
            new_file.write(line)